In [ ]:
%load_ext autoreload
%autoreload 2

<p style="font-size:32px; font-weight: bolder; text-align: center">Equivariant ML models</p>
<p style="font-size:16px; text-align: center">A tutorial introduction</p>
<p style="text-align: center"><i> authored by: <a href="mailto:michele.ceriotti@gmail.com"> Michele Ceriotti </a></i></p>

_Equivariance_ indicates the property of a function for which the inputs and outputs are subject to the action of the same symmetries, and which commutes with the application of the symmetries, that is: $f(\hat{S}A) = \hat{S} f(A)$. _Invariance_ can be seen as a special case, in which  $f(\hat{S}A) = f(A)$.

This notebook provides a brief introduction to the concept of equivariance in the context of atomic-scale machine-learning models, focusing in particular on the case of 3D rotations and inversion - in technical terms the $O(3)$ group symmetries - and their combination with translations - the three-dimensional Euclidean group $E(3)$. 

# The tools of the trade

We will use a number of packages, including both general-purpose ones and some that are designed specifically to the construction and manipulation of equivariant ML models. You should be able to fetch most of these from `pip`, while others might require a custom installation.

Besides some time-tested classics...

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

... we will use [ASE](https://wiki.fysik.dtu.dk/ase/) to load and manipulate atomic structures 

In [ ]:
import ase.io

... [chemiscope](https://chemiscope.org) to visualize them, [equistore](https://github.com/lab-cosmo/equistore) to hold equivariant quantities and [rascaline](https://github.com/Luthaf/rascaline) to compute atom-centered density correlations (ACDCs) - the main family of equivariant descriptors we will use.

In [ ]:
import equistore
from rascaline import SphericalExpansion
import chemiscope
import warnings # ignore chemiscope warnings
warnings.filterwarnings(action='ignore', module='chemiscope')

In [ ]:
# uncomment these if you miss some of the packages
# !pip install chemiscope
# !pip install --upgrade https://github.com/Luthaf/temporary-wheels/raw/main/equistore-74e0eabb-py3-none-linux_x86_64.whl
# !pip install --upgrade https://github.com/Luthaf/temporary-wheels/raw/main/rascaline-17b1cea7-py3-none-linux_x86_64.whl
# !pip install ase 

## Water dataset

We use as a demonstrative dataset a simple dataset that contains a water molecule, stretched and bent in different ways. For each configuration, the dataset holds energy, forces and dipole moments, computed using the high-accuracy [Partridge-Schwenke fits](10.1063/1.473987). 

In [ ]:
frames = ase.io.read("data/chemrev_nuprime-theta-grid_computed.xyz", ":")

In [ ]:
cs = chemiscope.show(frames, 
        settings={
            'map' : { 'x':{'property' : "HOH"},  'y':{'property' : "OH1"}, 'color' : {'property' : 'energy'} },
            'structure': [{'axes': 'xyz','keepOrientation': True}]
        })
display(cs)

Processes the properties of the structures (as stored in the `.extxyz` file) as `TensorBlock` objects. We will see later how these can be used. 

In [ ]:
energy_values = []
force_values = []
dipole_values = []
for f in frames:
    energy_values.append(f.info['energy'])
    dipole_values.append(f.info['dipole'])
    force_values.append(f.arrays['force'])

In [ ]:
energies = equistore.TensorBlock(values=np.asarray(energy_values).reshape(-1,1,1), 
                                 samples=equistore.labels.Labels(names = ["structure"], values=np.arange(len(frames), dtype=np.int32).reshape(-1,1) ),
                                 components=[equistore.labels.Labels.single()],
                                 properties=equistore.labels.Labels.single(),
                                )

energies.add_gradient("positions", data=-np.array(force_values).reshape(-1,3,1,1),
                      samples=equistore.labels.Labels(names=["sample", "atom"], 
                                                      values=np.asarray(np.vstack([ [[i,0],[i,1],[i,2]] for i in range(len(frames))]),dtype=np.int32) ),
                      components=[equistore.labels.Labels(names=["gradient_direction"], values=np.asarray([[0,1,2]], dtype=np.int32).T),
                                  equistore.labels.Labels.single()
                                 ]
                     )

In [ ]:
dipoles = equistore.TensorBlock(values=np.asarray(dipole_values).reshape(-1,3,1), 
                                 samples=equistore.labels.Labels(names = ["structure"], values=np.arange(len(frames), dtype=np.int32).reshape(-1,1) ),
                                 components=[equistore.labels.Labels(names=["direction"], values=np.asarray([[0,1,2]], dtype=np.int32).T) ],
                                 properties=equistore.labels.Labels.single(),
                                )

# Rotations and vectors

Let's now start looking into how a vector such as the dipole moment rotates. The key concept is that a vectorial property such as the atomic positions or the dipole moment transform, under the action of a rotation operation $\hat{R}$ in a way consistent with the application of a rotation matrix $\mathbf{R}$, i.e. if a structure $A$ has atomic positions $\mathbf{r}_i$ and dipole moment $\mathbf{y}_i$ (each of these being a 3-vector corresponding to the Cartesian coordinates $(x,y,z)$) then the rotated structure $\hat{R}A$ has atomic coordinates $\mathbf{R}\mathbf{r}_i$ and dipole moment $\mathbf{R}\mathbf{y}$. 


<img src="figures/rotations.png" width="400"/>

A rotation can be defined in terms of [Euler angles](https://en.wikipedia.org/wiki/Euler_angles), a set of three angles $(\alpha, \beta, \gamma)$ that define the orientation of a rigid body relative to a reference frame.  This is a problem that is made quite tricky by the existence of dozen of alternative conventions. For those in the know, we use the $ZYZ$ intrinsic rotations definition, which is also the one commonly used to define quantities in angular momentum theory. We use a wrapper from some utilities that returns the rotation matrix in the "correct" format, and use it to generate a set of water molecules in which the position of the atoms and the molecular dipoles have been rotated appropriately. 

In [ ]:
from utils.rotations import rotation_matrix, xyz_to_spherical, spherical_to_xyz, wigner_d_real
from utils.clebsh_gordan import ClebschGordanReal

In [ ]:
rotated_structures = []
rotated_wdipole = []
selected_frame = frames[81]
selected_dipole = dipoles.values[81,:,0]
# adds two fake atoms to show the magnitude and direction of the dipole moment
dipole_mol = ase.Atoms("FH", positions=[[0,2,0],selected_dipole+[0,2,0]])
for alpha in np.linspace(0, 2*np.pi, 8):
    for beta in np.linspace(0, np.pi, 4):
        for gamma in np.linspace(0, 2*np.pi, 8):
            rot_frame = selected_frame.copy() 
            rot_frame.info['alpha'] = alpha
            rot_frame.info['beta'] = beta
            rot_frame.info['gamma'] = gamma
            # rotates the frame
            R = rotation_matrix(alpha, beta, gamma)
            rot_frame.positions = rot_frame.positions@R.T
            rot_frame.cell = rot_frame.cell@R.T
            rot_frame.info['dipole'] = rot_frame.info['dipole']@R.T
            rotated_structures.append(rot_frame)
            
            # just for visualization, frames with dipole visualized as HF
            rot_frame = selected_frame.copy() + dipole_mol
            rot_frame.positions = rot_frame.positions@R.T
            rot_frame.cell = rot_frame.cell@R.T
            rot_frame.info.update(dict(alpha=alpha, beta=beta, gamma=gamma))
            rotated_wdipole.append(rot_frame)

In [ ]:
cs = chemiscope.show(rotated_wdipole, 
        settings={
            'map' : {
                'x' : {'property' : 'alpha'},
                'y' : {'property' : 'beta'},
                'z' : {'property' : 'gamma'},
                'color' : {'property' : 'energy'} },
            'structure': [{'axes': 'xyz','keepOrientation': True}]
        })
display(cs)

<a id="spherical-tensors"> </a>

# Rotations and arbitrary tensors

Vectorial quantities such as the dipole $\mathbf{y}$ are just one of the possible types of equivariant properties associated with a molecular structure. We consider as an example the tensor

$$
\mathbf{Y} = \mathbf{y}\mathbf{y}^T + |\mathbf{y}|^2 \mathbf{1},  
$$

built starting from the dipole moment. 

In [ ]:
def y_to_Y(y):
    return y.reshape(-1,1)@y.reshape(1,-1) + (y@y)*np.eye(3)

In [ ]:
for rot_frame in rotated_structures:
    rot_frame.info['Y'] = y_to_Y(rot_frame.info['dipole'])

This quantity transforms under rotation as $\mathbf{Y}(\hat{R}A) = \mathbf{R}\mathbf{Y}(A)\mathbf{R}^T$. 
While this is a perfectly fine way of expressing the transformation, it is more complicated than necessary: all the $9$ components of $\mathbf{Y}$ are formally mixed in the process, and even though it is linear in $\mathbf{Y}$, to explicitly cast it as such requires combining the two multiplications by $\mathbf{R}$ into the multiplication by a large, $9\times 9$ matrix.

In [ ]:
# picks one of the rotated structures
base = rotated_structures[0]
selected = rotated_structures[11]
base_Y = base.info['Y']
selected_Y = selected.info['Y']
# builds the rotation matrix
R = rotation_matrix(selected.info['alpha'], selected.info['beta'], selected.info['gamma'])

In [ ]:
print(base_Y)
# rotated Y matches the Y of the rotated structure
print(R@base_Y@R.T)
print(selected_Y)

In [ ]:
# builds the 9x9 rotation matrix - can you follow the index juggling?
flat_idx = np.array([(0,0),(0,1),(0,2),(1,0),(1,1),(1,2),(2,0),(2,1),(2,2)], dtype=int)
RR = R[flat_idx[:,0]][:,flat_idx[:,0]]*R[flat_idx[:,1]][:,flat_idx[:,1]]

In [ ]:
# the 9x9 matrix is almost entirely full
plt.matshow(RR, cmap=plt.cm.RdBu, vmin=-max(np.abs(RR.flatten())), vmax=max(np.abs(RR.flatten())))

In [ ]:
# the 9x9 matrix rotates the tensor with a single multiplication
(RR@(base_Y[flat_idx[:,0],flat_idx[:,1]])).reshape(3,3)

This brings us to the concept of _irreducible_ representations of the rotation group. In very hand-wavy terms, we look for ways to simplify the transformation by recombining entries in $\mathbf{Y}$ in such a way that they are less mixed up when acted upon by a rotation $\hat{R}$. 
That this might be possible can be seen by noticing that the _trace_ of $\mathbf{Y}$, $Y_{xx}+Y_{yy}+ Y_{zz}$ is left unchanged by the rotation: it is _invariant_!

In [ ]:
print(np.trace(base_Y), np.trace(selected_Y))

It turns out that products of Cartesian coordinates can be rearranged into blocks that transform under rotation as _spherical harmonics_, $Y^m_l$. The theory is not entirely trivial, and gets even messier if (as we do here) one wants to express everything in terms of _real_ quantities. 
The manipulations here are somewhat opaque (and rely heavily on utility functions) but allow to follow the construction of the _spherical tensor_ form of $\mathbf{Y}$. 

In [ ]:
# we need Clebsch-Gordan coefficients to transform the Cartesian tensor into an irreducible form
cg = ClebschGordanReal(l_max=2)
# xyz_to_spherical transforms Cartesian coordinates into a form consistent 
# with the definition of l=1 real spherical harmonics we use 
cg.couple(xyz_to_spherical(base_Y))

The output indicates that the tensor was built as the product of two vector-like ($l=1$) spherical Harmonics, and it has been decomposed into a scalar ($l=0$), pseudo-vector ($l=1$, zero because the tensor is symmetric), and higher-order ($l=2$) term. Each block contains $2m+1$ components. We can therefore stack these in a a single array with 9 components, that could (in principle) be written as a unitary transformation of the initial Cartesian form.

In [ ]:
def to_coupled(cartesian):
    coupled = cg.couple(xyz_to_spherical(cartesian))
    return np.hstack( [coupled[(1,1)][l] for l in [0,1,2]] )
base_coupled_Y = to_coupled(base_Y)
selected_coupled_Y = to_coupled(selected_Y)
# note that elements [1:4] are zero: this is a consequence of the symmetry of Y
print(base_coupled_Y)
print(selected_coupled_Y)

Compare the $9\times 9$ matrix above with that associated with rotating the tensor in this new basis. Each of the irreducible blocks transform separately, so the rotation matrix is block-diagonal. _Any_ Cartesian tensor can be expressed in terms of a collection of $Y^m_l$-like irreducible blocks.

In [ ]:
# the rotation matrix for the irreducible form can be built from a collection
# of (real-valued) Wigner D matrices
coupled_RR = np.zeros((9,9))
coupled_RR[0,0] = wigner_d_real(0, selected.info['alpha'], selected.info['beta'], selected.info['gamma'])
coupled_RR[1:4,1:4] = wigner_d_real(1, selected.info['alpha'], selected.info['beta'], selected.info['gamma'])
coupled_RR[4:9,4:9] = wigner_d_real(2, selected.info['alpha'], selected.info['beta'], selected.info['gamma'])

In [ ]:
# the 9x9 matrix is block diagonal
plt.matshow(coupled_RR, cmap=plt.cm.RdBu, vmin=-max(np.abs(RR.flatten())), vmax=max(np.abs(RR.flatten())))

In [ ]:
# this block-diagonal matrix performs a rotation in the irreducible spherical basis
print(base_coupled_Y)
print(coupled_RR@base_coupled_Y)
print(selected_coupled_Y)

The transformation can be reverted to bring the rotated tensor back to its Cartesian form

In [ ]:
coupled = {(1,1): {0:selected_coupled_Y[:1], 1:selected_coupled_Y[1:4], 2: selected_coupled_Y[4:9]}}
spherical_to_xyz(cg.decouple(coupled))

# Density expansion coefficients

We now move on to discuss how to describe an atomic environment using expansion coefficients of the atomic density. In short, the idea is to start describing a molecule $A$ in terms of localized functions (e.g. Gaussians) centered on each atom $i$, "labelled" by their chemical nature $a$

$$
\langle a \mathbf{x} | A; \rho\rangle = \sum_{i \in A} \delta_{a a_i} \langle \mathbf{x} | \mathbf{r}_i \rangle.
$$

We use  the notation $\langle \mathbf{x} | \mathbf{r}_i \rangle = g(\mathbf{x}-\mathbf{r}_i)$ to emphasize how the full structure is built as a sum of terms that describe individual atoms, and how this description can be implemented in any continuous or discrete basis. 
In general terms, in analogy with the Dirac notation used to describe a quantum state, we use the notation $\langle q | A\rangle$ to indicate a descriptor $| A\rangle$ for an entity $A$, discretized in a basis that is enumerated by the index $q$. 
See Section 3.1 of [this review](https://doi.org/10.1021/acs.chemrev.1c00021) for a gentler introduction. 

This density is then symmetrized with respect to translations (reflecting the fact that atomic properties are invariant to rigid translations of a molecule) which leads to expressing the structure descriptors as a sum of descriptors of _atom centered environments_ $A_i$,


$$
\langle a \mathbf{x} | A; \rho\rangle = \sum_i \langle a \mathbf{x} | \rho_i\rangle
$$

$$
\langle a \mathbf{x} | \rho_i\rangle = \sum_{j \in A_i} \delta_{a a_j} \langle \mathbf{x} | \mathbf{r}_{ji} \rangle.
$$

where the Gaussians are evaluated at the interatomic distance vectors $\mathbf{r}_{ji}=\mathbf{r}_j-\mathbf{r}_i$.

To manipulate this atom-centered density, it is more convenient to express it on a discrete basis. Guided by symmetry considerations that will become apparent, and in analogy with what is done routinely in quantum chemistry for the electron wafefunction (or density) we use a basis of radial functions $R_{nl}(x) \equiv \langle x||nl\rangle$ and spherical harmonics $Y^m_{l}(\hat{\mathbf{x}}) \equiv \langle \hat{\mathbf{x}}|lm\rangle$

$$
\langle a nlm | \rho_i\rangle = \int \mathrm{d}\mathbf{x} 
 \langle nl| x\rangle  \langle lm| \hat{\mathbf{x}} \rangle
\langle a \mathbf{x} | \rho_i\rangle  
$$

## Computing descriptors with _rascaline_ and _equistore_

We use `rascaline` to compute the density expansion coefficients for these water molecule structures. 
The coefficients are returned as a `equistore.TensorMap` object, that reflects some of the considerations discussed above on the structure of atom-centered density features

In [ ]:
hypers = {
    "cutoff": 2.0,
    "max_radial": 6,
    "max_angular": 4,
    "atomic_gaussian_width": 0.2,
    "radial_basis": {"Gto": {}},
    "cutoff_function": {"ShiftedCosine": {"width": 0.5}},
    "center_atom_weight": 1.0,
    "gradients": False,
}

calculator = SphericalExpansion(**hypers)

descriptor = calculator.compute(frames)

A `TensorMap` object works as a container that holds blocks of data. The pattern is reminiscent of a `dict`, but with some more structure and metadata: each block is associated with a _key_, which consists in a tuple of ints. The set of keys is a `equistore.Labels` object, that also keeps track of the _names_ that describe each index in the key. 

The expansion keys hold the `spherical_harmonics_l` index, and two indices corresponding to the atomic number of the central atom `species_center` and of the neighbors `species_neighbor`. 

In [ ]:
descriptor.keys

Each block is associated to a `equistore.TensorBlock` object. Each block is associated with a dense tensor with a `samples` direction (enumerating the items that are described) a `properties` direction (enumerating actual properties, or descriptors) and zero or more `components` - typical examples would be the Cartesian coordinates of a vector, or the $m$ index in spherical harmonics. Each is associated with a `Labels` object that stores metadata that describe the entries.

In [ ]:
# this is l=2, central atom is O and neighbor is H
block = descriptor.block(spherical_harmonics_l=2,species_center=8,species_neighbor=1)

In [ ]:
block.samples

In [ ]:
block.properties

In [ ]:
block.components

Depending on the type of system or application, some indices might be stored more conveniently as samples, properties, components, or as sparse keys. _equistore_ provides utility functions to reorder the data for more convenient manipulation. For instance, in this case all atoms have the same type of neighbors, and so it does not make sense to use sparse storage for the neighbor element index. We can move the index from keys to properties:

In [ ]:
descriptor.keys_to_properties('species_neighbor')

In [ ]:
descriptor.keys

In [ ]:
descriptor.block(0).properties

## Equivariance of the descriptors

Let's now see how the density coefficients actually behave as equivariant descriptors. This is kind of obvious given that we build them by expanding the density on a basis of spherical harmonics! Here we compute descriptors for two of the rotated structures, and see how the same result can be achieved by applying the rotation to the base molecule using a Wigner-D matrix.

In [ ]:
base_descriptor = calculator.compute([base])
selected_descriptor = calculator.compute([selected])
base_descriptor.keys_to_properties('species_neighbor')
selected_descriptor.keys_to_properties('species_neighbor')

In [ ]:
# pick just one block as example
base_block = base_descriptor.block(spherical_harmonics_l=3, species_center=8)
selected_block = selected_descriptor.block(spherical_harmonics_l=3, species_center=8)

In [ ]:
selected_block.values

In [ ]:
wd = wigner_d_real(3, selected.info['alpha'], selected.info['beta'], selected.info['gamma'])

In [ ]:
wd@base_block.values[0]

In light of this, we can see density coefficients as _symmetry adapted_ descriptors. To emphasize their equivariant behavior, and that they reflect the distribution of neighbors by summing over them one of a time, we indicate the coefficients as 

$$
\langle an|\overline{\rho_i^{\otimes 1}; \lambda \mu }\rangle \equiv
\langle an\lambda \mu|\rho_i\rangle
$$

# Atom-centered density correlations

The density expansion coefficients can be regarded as the simplest possible form of an equivariant descriptor built based on the neighbor density. The invariant part, $\langle an|\overline{\rho_i^{\otimes 1}; 00 }\rangle$ corresponds to a discretization of the pair correlation function: using a real-space basis,

$$
    \langle ax|\overline{\rho_i^{\otimes 1}; 00 }\rangle \approx \sum_{j\in A_i} \delta_{a a_j} \langle x | r_{ji} \rangle 
$$

where $ \langle x | r_{ji} \rangle $ is a localized function centered on $r_{ji}$. 

In order to obtain a richer description of the atomic environment it is possible to combine several copies of $\langle a\mathbf{x} | \rho_i \rangle$, to build $\nu$-neighbors atom-centered density correlations (ACDCs). 
The formalism we use was introduced by [Willatt et al.](https://doi.org/10.1063/1.5090481), and is explained in detail, discussing its relation with the leading frameworks for atomistic machine learning, in a [review by Musil et al.](https://doi.org/10.1021/acs.chemrev.1c00021)

Essentially, the idea is that considering tensor products of the atom density provides simultaneous information on the mutual position of several neighbors

$$
\langle \mathbf{x} |  \rho_i \rangle \langle \mathbf{x}' |  \rho_i \rangle =
\sum_{jj'\in A_i}
\langle \mathbf{x} |\mathbf{r}_{ji} \rangle \langle \mathbf{x}' |\mathbf{r}_{j'i} \rangle. 
$$

Much like for the case of [Cartesian tensors](#spherical-tensors), these tensor products can be re-cast in terms of irreducible representations of the O(3) (rotations + inversion) group. A (comparatively) simple and efficient way to obtain these equivariant ACDC features is to build them iteratively, as discussed e.g. by [Nigam et al. ](https://doi.org/10.1063/5.0021116), using an expression that corresponds to the combination of angular momenta in quantum mechanics using Clebsch-Gordan coefficients $\langle k m_1\  l m_2 | \lambda \mu \rangle$.

$$
\langle q k;  a n l|\overline{\rho_i^{\otimes (\nu+1)}; \sigma (-1)^{l+k+\lambda};  \lambda \mu } \rangle = 
\sum_{m_1 m_2} 
\langle q |\overline{\rho_i^{\otimes \nu}; k m_1 } \rangle
\langle a n|\overline{\rho_i^{\otimes 1}; \sigma; l m_2 } \rangle 
\langle k m_1\  l m_2 | \lambda \mu \rangle.
$$

The additional index $\sigma$ in the ket $|\sigma; \lambda \mu>$ takes the values $\pm 1$ and tracks the parity of the descriptors with respect to inversion (all the $\nu=1$ equivariants have a parity $+1$, which means that they pick up a phase $(-1)^\lambda$ under inversion).

## Computing $\nu=2$ equivariants ($\lambda$-SOAP)

This is a lot to take in, so let's see how this is realized in a practical case. First, we convert the descriptors in a standardized form (the combination routines use the metadata in TensorMap to generate the right combinations).

In [ ]:
from equistore import Labels, TensorBlock, TensorMap
from utils.acdc_mini import *

In [ ]:
# precompute the C-G coefficients so they can be reused. we need to go up to l_max*2 
cg = ClebschGordanReal(l_max=hypers["max_angular"])

In [ ]:
# this enforces a consistent naming convention for the various components
acdc_nu1 = acdc_standardize_keys(descriptor) 

In [ ]:
# and this makes sure that all blocks have the same size, 
# by merging blocks associated with different central atoms
acdc_nu1.keys_to_samples("species_center")

In [ ]:
acdc_nu1.keys

In [ ]:
acdc_nu1.block(spherical_harmonics_l=1).properties

... then we call the C-G combination using a utility subroutine. The combinations generate more blocks (some with even and some with odd inversion parity, and potentially up to twice the maximum $l$ in the inputs), and for each block many more features (the full tensor product of species and radial functions, as well as angular momentum labels). This function computes ALL combinations (up to an equivariant cutoff `lcut`), which is very wasteful because many combinations are linearly dependent.

In [ ]:
acdc_nu2 = cg_increment(acdc_nu1, acdc_nu1, clebsch_gordan=cg, lcut=hypers["max_angular"])

In [ ]:
acdc_nu2.keys

In [ ]:
acdc_nu2.block(0).properties.names

## Equivariance of the higher-order descriptors

Let's now see how the density coefficients actually behave as equivariant descriptors. This is kind of obvious given that we build them by expanding the density on a basis of spherical harmonics! Here we compute descriptors for two of the rotated structures, and see how the same result can be achieved by applying the rotation to the base molecule using a Wigner-D matrix.

In [ ]:
test_descriptors = calculator.compute([base, selected])
test_descriptors.keys_to_properties('species_neighbor')
test_nu1 = acdc_standardize_keys(test_descriptors) 
test_nu1.keys_to_samples("species_center")
test_nu2 = cg_increment(test_nu1, test_nu1, clebsch_gordan=cg, lcut=hypers["max_angular"])

In [ ]:
test_block = test_nu2.block(spherical_harmonics_l=3, inversion_sigma=1)

In [ ]:
# for simplicity, we are computing all descriptors in one go, 
# so we fetch the block corresponding to the base and the rotated O
test_base = test_block.values[test_block.samples.position((0,0,8))]
test_selected = test_block.values[test_block.samples.position((1,0,8))]

Recall that `selected` is a rigidly-rotated copy of the `base` molecule, so the features should correspond after being rotated with an appropriate Wigner matrix...

In [ ]:
wd = wigner_d_real(3, selected.info['alpha'], selected.info['beta'], selected.info['gamma'])

In [ ]:
test_selected

In [ ]:
wd@test_base   # ... and they do match!

# An equivariant ML model

Now, let's see how we can use equivariant features to build an equivariant model that predicts the dipole of the water molecule. We will do so with linear regression, that reveals clearly the nature of the problem. 
It is interesting to note that the algebraic structure of $O(3)$ means that in practice most equivariant models have a structure that is very close to that underlying the iterative construction of ACDCs, combined with linear regression. 
[Cormorant](https://proceedings.neurips.cc/paper/2019/file/03573b32b2746e6e8ca98b9123f2249b-Paper.pdf) and [Tensor Field Networks](http://arxiv.org/abs/1802.08219v3) are two examples of equivariant networks that effectively build linear combinations of $|\overline{\rho_i^{\otimes \nu}; \sigma;  \lambda \mu }\rangle$ features, and pretty much all other schemes can be regarded as injecting non-linearities in the form of multipliers that depend only on scalar descriptors. See [Nigam et al.](https://aip.scitation.org/doi/10.1063/5.0087042) for a formal discussion of the analogies and differences between ACDCs and alternative schemes.

A linear model for a tensorial property $y^{\sigma;\lambda}_\mu$ (already expressed in its irreducible spherical form) can be readily built as 

$$
\tilde{y}^{\sigma;\lambda}_\mu(A_i) \approx \sum_q w_q \langle q|\overline{\rho_i^{\otimes \nu}; \sigma;  \lambda \mu }\rangle.
$$

It is essential that the weights depend on the feature weight, but not on the symmetry component $\mu$: then,

$$
\tilde{y}^{\sigma;\lambda}_\mu(\hat{R}A_i) \approx \sum_q w_q \langle q|\hat{R}\overline{\rho_i^{\otimes \nu}; \sigma;  \lambda \mu }\rangle = \hat{R}y^{\sigma;\lambda}_\mu(A_i),
$$

provided that $|\overline{\rho_i^{\otimes \nu}; \sigma;  \lambda \mu }\rangle$ is equivariant to $\hat{R}$. 

By defining a $L^2$ loss in which one sums over the mean square error for all components $\mu$ as well as the train set indices, 

$$
\ell = \sum_{i\mu} |\tilde{y}^{\sigma;\lambda}_\mu(A_i) - y^{\sigma;\lambda}_\mu(A_i)|^2
$$

one sees that in order to build a linear regression scheme it is enough to treat the symmetry components as sample indices -- although one must pay attention to keep blocks associated with a given structure together when splitting the data set or performing cross-validation. This means one can use standard ridge regression from `sklearn`. Note it's important _not_ to fit an intercept, as that is incompatible with rotational symmetry: all $\lambda>0$ tensor components must average to zero over rotations.

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
ridge = Ridge(alpha = 1e-8, fit_intercept = False)

In [ ]:
# uses only O as centers (simpler, there's just one sample per molecule)
idx = np.where(acdc_nu1.block(inversion_sigma=1, spherical_harmonics_l=1).samples["species_center"] == 8)[0]
# try using acdc_nu1 as descriptors: performances are horrible because it does not hold info on the HOH angle!
X = acdc_nu2.block(inversion_sigma=1, spherical_harmonics_l=1).values[idx]
# for a vector, this is all we need to go into spherical form
y = xyz_to_spherical(dipoles.values)
ridge.fit(X[::2].reshape(-1,X.shape[-1]), y[::2].flatten())

In [ ]:
yp = ridge.predict(X.reshape(-1,X.shape[-1]) ).reshape(-1,3)

In [ ]:
plt.plot(y[1::2].flatten(),yp[1::2].flatten(),'r.', label="test")
plt.plot(y[::2].flatten(),yp[::2].flatten(),'b.', label="train")
plt.xlabel(r"$y_\mu$ / eV/Å")
plt.ylabel(r"$\tilde{y}_\mu$ / eV/Å")
plt.legend();

Even though all structures in the train set lie in the $xy$ plane, the model predicts perfectly well the dipoles of arbitrarily oriented molecules.

In [ ]:
rotated_descriptors = calculator.compute(rotated_structures)
rotated_descriptors.keys_to_properties('species_neighbor')
rot_nu1 = acdc_standardize_keys(rotated_descriptors) 
rot_nu1.keys_to_samples("species_center")
rot_nu2 = cg_increment(rot_nu1, rot_nu1, clebsch_gordan=cg, lcut=hypers["max_angular"])

In [ ]:
idx = np.where(rot_nu1.block(inversion_sigma=1, spherical_harmonics_l=1).samples["species_center"] == 8)[0]
X = rot_nu2.block(inversion_sigma=1, spherical_harmonics_l=1).values[idx]
rot_yp = spherical_to_xyz(ridge.predict(X.reshape(-1,X.shape[-1]) ).reshape(-1,3))

In [ ]:
rot_y = np.asarray([frame.info['dipole'] for frame in rotated_structures])

In [ ]:
plt.plot(rot_y.flatten(), rot_yp.flatten(), 'b.')
plt.xlabel(r"$y_\mu$ / eV/Å")
plt.ylabel(r"$\tilde{y}_\mu$ / eV/Å")
plt.xlabel(r"$y_\alpha$ / eV/Å")
plt.ylabel(r"$\tilde{y}_\alpha$ / eV/Å")

Let's look at how the predicted dipoles rotate with the molecule...

In [ ]:
rotated_wdipolep = []
dipole_mol = ase.Atoms("FH", positions=[[0,2,0],selected_dipole+[0,2,0]])
for i,f in enumerate(rotated_structures):
    # for visualization purpose we display the dipole as an HF molecule - the origin must also be rotated, which we had done manually above
    rotated_wdipolep.append(f+ase.Atoms("FH", 
        positions=[rotated_wdipole[i].positions[3],rotated_wdipole[i].positions[3]+rot_yp[i]*2])
                           )

In [ ]:
cs = chemiscope.show(rotated_wdipolep, 
        settings={
            'map' : {
                'x' : {'property' : 'alpha'},
                'y' : {'property' : 'beta'},
                'z' : {'property' : 'gamma'},
                'color' : {'property' : 'energy'} },
            'structure': [{'axes': 'xyz','keepOrientation': True}]
        })
display(cs)